# Forced Alignment Demo

This notebook demonstrates how to align an audio file with a phonemized transcript using `torchaudio.functional.forced_align` and the `facebook/wav2vec2-xlsr-53-espeak-cv-ft` model.

In [ ]:
import torchimport torchaudioimport refrom transformers import Wav2Vec2FeatureExtractor, Wav2Vec2ForCTCfrom transformers.models.wav2vec2_phoneme.tokenization_wav2vec2_phoneme import Wav2Vec2PhonemeCTCTokenizerfrom IPython.display import Audio, display

In [ ]:
#@title Load audio and transcriptwav_path = 'path/to/audio.wav'  #@param {type:'string'}transcript = 'a n d dh e r w iː g oʊ'  #@param {type:'string'}def normalize(text):    cleaned = []    for tok in text.split():        tok = re.sub(r'[!.,?"«»]', '', tok)        if not tok:            continue        if tok in tokenizer.get_vocab():            cleaned.append(tok)        else:            print(f'Skipping unsupported token: {tok}')    return ' '.join(cleaned)waveform, sr = torchaudio.load(wav_path)feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('facebook/wav2vec2-xlsr-53-espeak-cv-ft')tokenizer = Wav2Vec2PhonemeCTCTokenizer.from_pretrained('facebook/wav2vec2-xlsr-53-espeak-cv-ft', espeak_path='espeak')model = Wav2Vec2ForCTC.from_pretrained('facebook/wav2vec2-xlsr-53-espeak-cv-ft')model.eval()transcript = normalize(transcript)target_sr = feature_extractor.sampling_rateif sr != target_sr:    waveform = torchaudio.functional.resample(waveform, sr, target_sr)with torch.no_grad():    inputs = feature_extractor(waveform.squeeze(), sampling_rate=target_sr, return_tensors='pt')    logits = model(inputs.input_values).logits    emission = torch.log_softmax(logits, dim=-1)ids = tokenizer(transcript, add_special_tokens=False).input_idsif emission.size(1) < len(ids):    raise ValueError('Transcript too long for audio')tokens = torch.tensor([ids], dtype=torch.int32)alignments, scores = torchaudio.functional.forced_align(emission, tokens, blank=tokenizer.pad_token_id)alignments = alignments[0]

In [ ]:
# Display each aligned segment with playback
labels = tokenizer.convert_ids_to_tokens(range(len(tokenizer)))
frame_dur = feature_extractor.chunk_length / feature_extractor.sampling_rate
for frame, idx in enumerate(alignments.tolist()):
    label = labels[idx]
    start = frame * frame_dur
    end = start + frame_dur
    segment = waveform[:, int(start*target_sr):int(end*target_sr)]
    print(f'{frame:03d} {label} {start:.2f}-{end:.2f}')
    display(Audio(segment.squeeze().numpy(), rate=target_sr))